# Centering template, story and recall vectors
## 12/3/20; 12/9/20
## Centering everything usable together here

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as py
import plotly
import os
import glob
import math
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import copy
from random import randrange
from sklearn.metrics import jaccard_score
import random


from sklearn.decomposition import PCA #for cluster analysis
from gensim.models import KeyedVectors #for word embeddings
import matplotlib.pyplot as plt #for plotting
import os #for importing
import pickle #for loading transcripts
from scipy.stats import pearsonr 

# from _DRAFT_20200604_functions import * #includes constants and score function
from tqdm import tqdm_notebook #for progress bar

%autosave 5

Autosaving every 5 seconds


# 0. Importing story and template vectors and sums 

In [3]:
templates = pickle.load( open( 'template_vectors', "rb" ) )
stories = pickle.load( open( 'actual_story_vectors', "rb" ) )
recalls = pickle.load( open( 'recall_vectors', "rb" ) )
# sums
sums = pickle.load( open( "new_sums", "rb" ) )

# I. Center
## A. Plot all participants for each story - without centering
### How do  I plot a legend???
### 1. Concatenatate word vectors of template vectors

In [4]:
# Concatenating wvs for plotting below
wv_dim = 300
concat_templates = np.zeros((8, wv_dim))
order_keys = np.zeros((8,1))
count = 0
for key in templates:
    concat_templates[count, :] = templates[key]
    order_keys[count] = key
    count += 1

### 2. Concatenate all recall vectors

In [5]:
first = 0

keys = [11, 12, 13, 14, 21, 22, 23, 24, 31, 32, 33, 34, 41, 42, 43, 44]
ranges = [8]

concat_recalls = np.zeros((0,300))

for key in keys:
    for i in range(0, 3):
        n_partc = recalls[key][i].shape[0]
        for j in range(0, n_partc):
            concat_recalls = np.vstack((concat_recalls, recalls[key][i][j]))

### 3.Concatenate story vectors

In [6]:
first = 0

keys = [11, 12, 13, 14, 21, 22, 23, 24, 31, 32, 33, 34, 41, 42, 43, 44]
ranges = [8]



for key in stories:
    if first == 0:
        this = stories[key]
        concat_stories = this.reshape(this.shape[0],-1).T
        first = 1
    else:
        this = stories[key]
        this = this.reshape(this.shape[0],-1).T
        concat_stories = np.concatenate((concat_stories, this), axis =0)

### 4. Concatenate template, recall, and story vectors

In [7]:
all_wv = np.concatenate((concat_templates, concat_recalls, concat_stories), axis = 0)

### 5. Calculate centering vec by averaging

In [8]:
centering_vec = np.mean(all_wv, axis = 0)

### 6. Subtract the centering vec from template, recall, and story vectors

In [9]:
# template vectors

for key in templates:
    templates[key] = templates[key] - centering_vec
    
# recall vectors

for key in recalls:
    for i in range(0, 3):
        n = recalls[key][i].shape[0]
        for j in range(0,n):
            recalls[key][i][j] = recalls[key][i][j] - centering_vec
            
#story vectors

#make new dict with int key instead of string
int_stories = {}
for key in stories:
    int_stories[int(key)] = stories[key] - centering_vec

### 7. Export the centered dicts

In [10]:
pickle.dump( templates, open( 'c_template', "wb" ) )
pickle.dump( recalls, open( 'c_recall', "wb" ) )
pickle.dump( int_stories, open( 'c_stories', "wb" ) )

In [11]:
c_recall

NameError: name 'c_recall' is not defined

In [25]:
int_stories

{11: array([-1.93833065e-02, -1.68636302e-03, -4.47929158e-03, -1.89012582e-03,
         1.45446416e-02, -1.07878222e-02,  7.86332042e-03, -1.94804765e-03,
         1.71178582e-03,  3.87380223e-03,  4.58015200e-03, -7.71420506e-03,
        -2.78319537e-03, -6.21596834e-03, -1.48309688e-03, -3.64730267e-03,
        -2.49281268e-03, -2.18520239e-04,  2.95774523e-03, -3.40005086e-03,
         3.15011464e-03, -9.00309518e-03,  5.55004817e-03, -1.58708480e-03,
        -8.31191910e-03,  5.56879876e-04,  3.55721800e-03,  1.63570824e-02,
         6.00644553e-03, -7.77115892e-03,  3.11003808e-03, -2.19194227e-03,
         8.46362137e-03, -1.22538617e-03,  7.32825117e-03,  7.24105216e-03,
        -2.35736927e-03,  1.52510090e-02, -5.49233409e-03, -1.57730647e-03,
        -5.14669575e-03,  7.85897371e-03, -6.63322120e-03,  7.31964340e-03,
         5.47744916e-03, -5.19877980e-03, -7.09116994e-03,  1.89310367e-03,
        -2.75317154e-03, -1.11987791e-02,  1.94768922e-03,  6.47167736e-03,
        